In [114]:
import psycopg2

def q2(conn):
  #TODO: option for T1,2,3
  cur = conn.cursor()
  query = '''
  select room, day, starting, ending, weeks from T1rooms;
  '''
  cur.execute(query)
  rows = cur.fetchall() #the rows is order by room, day, start_time, end_time
                        # so start_time of next row always >= start_time of previous
  store = dict()
  for room, day, start, end, wk in rows:
    if room in store.keys():
      store[room] += count_time((end-start), wk)   
#       print(room, day, start, end, wk)
#       print(count_time((end - start),wk)) 
    else:
      #first see this room
      #calculating total hour normally, and store data into dict 
#       print(room, day, start, end, wk)
#       print(count_time((end - start),wk))
      store[room] = count_time((end - start),wk)
      
  
  underused = 0
  for hours in store.values():
    if (hours/10) < 20:
      underused += 1
  
  # get total_rooms
  sum_query = '''
  select sum(count) from (
    (select count(*) from rooms where code ~* '^K-.*' group by id)) as Foo;
  '''
  cur.execute(sum_query)
  total_rooms = cur.fetchone()[0]
  
  #get how many rooms used in T?
  T_rooms_query = '''
  select count(*) from (
  (select distinct room from T1rooms)) as Foo; 
  '''#TODO: fix T123
  cur.execute(T_rooms_query)
  T_rooms = cur.fetchone()[0]
  
  #unused room + underused room / total_ rooms
  result = ((total_rooms - T_rooms) + underused) / total_rooms
  print("{:.1%}".format(result))
  
  
  
# def get_no_overlap(start, end, wk, classes):
#   final_wk = list(wk)
#   for old_start, old_end, old_wk in classes:
#     # if overlap
#     if old_start <= start and start < old_end:
      
      

# def extra_time(end, old_end):
#   extra = end - old_end
#   if extra <= 0:
#     return 0
#   else:
#     return extra
  
# return as hours for wk1-10
def count_time(time, wk):
  count = 0
  # dont care about wk11
  for i in range(10):
    if wk[i] == '1':
      count += 1
  
  #calculating time
  hr = time // 100 * count 
  extra = 0.5 if (time % 100 == 30) or (time % 100 == 70) else 0
  extra *= count
  
  # return as hours
  return hr + extra
  
def connect():
  try:
    conn = psycopg2.connect("dbname=a3 user=postgres password=chenqq227") #TODO: delete

    q2(conn)

    conn.close()
  except Exception as e:
    print("Cannot connect the database: ", e)


In [115]:
connect()

51.4%


In [55]:
288 / 508

0.5669291338582677

In [72]:
508 - 350

158

In [98]:
158+130

288

In [99]:
288 / 508

0.5669291338582677

In [1]:
x = [1]

In [4]:
x.insert(0, 2)

In [5]:
x

[2, 1, 1]